

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [46]:
# Basic Libraries
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# For Lasso
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Run OLS
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# For Random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

In [47]:
# Loading the dataset
df_rental = pd.read_csv('rental_info.csv')
df_rental.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401


In [48]:
# Adding information about rental duration into dataset
df_rental['rental_length'] = pd.to_datetime(df_rental['return_date']) - pd.to_datetime(df_rental['rental_date'])
df_rental['rental_length_days'] = df_rental['rental_length'].dt.days

In [49]:
# Adding dummy variables for deleted scenes & behind the scenes
df_rental['deleted_scenes'] = np.where(df_rental['special_features'].str.contains('Deleted Scenes'), 1, 0)
df_rental['behind_the_scenes'] = np.where(df_rental['special_features'].str.contains('Behind the Scenes'), 1, 0)

In [50]:
# Picking columns that relates with target feature
cols_to_drop = ["special_features", "rental_length", "rental_length_days", "rental_date", "return_date"]

In [51]:
# Spillting dataset into feature & target sets
X = df_rental.drop(cols_to_drop, axis = 1)
y = df_rental['rental_length_days']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 9)

In [52]:
# Creating a Lasso() model
lasso = Lasso(alpha = 0.3, random_state = 9)

# Training model & accessing the co-efficients
lasso.fit(X_train, y_train)
lasso_coef = lasso.coef_

#Performing feature selection by picking columns with positive co-efficients
X_lasso_train, X_lasso_test = X_train.iloc[:, lasso_coef > 0], X_test.iloc[:, lasso_coef > 0]

In [53]:
# Running OLS models on chosen regression
ols = LinearRegression()

ols = ols.fit(X_lasso_train, y_train)
y_test_pred = ols.predict(X_lasso_test)

mse_lin_reg_lasso = mean_squared_error(y_test, y_test_pred)
print(mse_lin_reg_lasso)

4.812297241276244


In [54]:
# Random forest hyperparameter space
param_dist = {'n_estimators': np.arange(1, 101, 1), 'max_depth' : np.arange(1, 11,1)}

In [55]:
# Creating a Random forest regressor
rf = RandomForestRegressor()

# Using random search to find the best hyperparameters
rand_search = RandomizedSearchCV(rf, param_distributions = param_dist, cv = 5, random_state = 9)

# Fitting the random search object to the data
rand_search.fit(X_train, y_train)

# Creating a variable for the best hyperparameters
hyper_params = rand_search.best_params_

In [56]:
# Running the random forest on the chosen hyperparameters
rf = RandomForestRegressor(n_estimators = hyper_params['n_estimators'], max_depth = hyper_params['max_depth'], random_state = 9)

rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
mse_random_forest = mean_squared_error(y_test, rf_pred)
print(mse_random_forest)

2.225667528098759


In [57]:
# Final Conclusion
best_model = rf
best_mse = mse_random_forest